In [1]:
import networkx as nx
import numpy as np
from collections import deque

**Exercise 3.2:** My implementation of ```reachable_nodes_bfs``` is efficient in the sense that it is in $\mathcal{O}(n+m)$, but it incurs a lot of overhead adding nodes to the queue and removing them. NetworkX provides a simple, fast implementation of BFS, available from the NetworkX repository on GitHub at http://thinkcomplex.com/conxx.

Here is a version I modified to return a set of nodes:

In [ ]:
def plain_bfs(G, start):
  
  seen = set()
  nextlevel = {start}
  while nextlevel:
    thislevel = nextlevel
    nextlevel = set()
    for v in thislevel:
      if v not in seen:
        seen.add(v)
        nextlevel.update(G[v])
  return seen

Compare this function to ```reachable_nodes_bfs``` and see which is faster. Then see if you can modify this function to implement a faster version of ```shortest_path_dijkstra```.



---
Answer


---

I will provide below the implementation of ```reachable_nodes_bfs```. I will also provide the implementation of ```make_ring_lattice``` from the book.



In [ ]:
def reachable_nodes_bfs(G, start):
  seen = set()
  queue = deque([start])
  while queue:
    node = queue.popleft()
    if node not in seen:
      seen.add(node)
      queue.extend(G.neighbors(node))
  return seen

In [ ]:
def adjacent_edges(nodes, halfk):
    n = len(nodes)
    for i, u in enumerate(nodes):
        for j in range(i+1, i+halfk+1):
            v = nodes[j % n]
            yield u, v

def make_ring_lattice(n, k):
    G = nx.Graph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from(adjacent_edges(nodes, k//2))
    return G

lattice = make_ring_lattice(1000,100)

We will know compare ```plain_bfs``` and ```reachable_nodes_bfs``` using ```%timeit```.

In [ ]:
%timeit len(plain_bfs(lattice,0))

100 loops, best of 3: 3.79 ms per loop


In [ ]:
%timeit len(reachable_nodes_bfs(lattice,0))

100 loops, best of 3: 8.89 ms per loop


Conclusion: The ```plain_bfs``` implementation is way better than ```reachable_nodes_bfs```.

We will now modify ```plain_bfs``` to make it slightly faster. One approach to it is by not using ```seen.add(v)```, especially inside a loop. Every time we call this function, we call a larger and larger set. As an example, see the code below.

In [ ]:
a = set()
for i in range(10):
  a.add(i)
  print(a)

{0}
{0, 1}
{0, 1, 2}
{0, 1, 2, 3}
{0, 1, 2, 3, 4}
{0, 1, 2, 3, 4, 5}
{0, 1, 2, 3, 4, 5, 6}
{0, 1, 2, 3, 4, 5, 6, 7}
{0, 1, 2, 3, 4, 5, 6, 7, 8}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


To remedy this, we could make an empty dictionary, instead of using an empty set and calling the whole set as we update it. By using the dictionary, we could instead add an element and its key without calling the whole dictionary. Below is the implementation of that idea.

In [ ]:
def plain_bfs_modified(G, start):
  key = 0
  box = {}
  nextlevel = {start}
  while nextlevel:
    thislevel = nextlevel
    nextlevel = set()
    for v in thislevel:
      if v not in box:
        box[v] = key
        nextlevel.update(G[v])
    key += 1
  return box

In [ ]:
%timeit len(plain_bfs_modified(lattice,0))

100 loops, best of 3: 3.77 ms per loop


In [ ]:
def shortest_path_dijkstra(G, source):
  dist = {source: 0}
  queue = deque([source])
  while queue:
    node = queue.popleft()
    new_dist = dist[node] + 1

    neighbors = set(G[node]).difference(dist)
    for n in neighbors:
      dist[n] = new_dist

    queue.extend(neighbors)
  return dist

In [ ]:
%timeit len(shortest_path_dijkstra(lattice,0))

100 loops, best of 3: 6.32 ms per loop


In [ ]:
%timeit len(nx.shortest_path_length(lattice,0))

100 loops, best of 3: 3.07 ms per loop


We can see that our implementation is better than that of ```shortest_path_dijkstra```.